In [1]:
from graph_of_thoughts.prompter import LanguageModelPrompter
from graph_of_thoughts.language_model import ChatGPT
lm = ChatGPT()
table = """ {
        "column_names": [
            [
                -1,
                "*"
            ],
            [
                0,
                "city id"
            ],
            [
                0,
                "official name"
            ],
            [
                0,
                "status"
            ],
            [
                0,
                "area km 2"
            ],
            [
                0,
                "population"
            ],
            [
                0,
                "census ranking"
            ],
            [
                1,
                "farm id"
            ],
            [
                1,
                "year"
            ],
            [
                1,
                "total horses"
            ],
            [
                1,
                "working horses"
            ],
            [
                1,
                "total cattle"
            ],
            [
                1,
                "oxen"
            ],
            [
                1,
                "bulls"
            ],
            [
                1,
                "cows"
            ],
            [
                1,
                "pigs"
            ],
            [
                1,
                "sheep and goats"
            ],
            [
                2,
                "competition id"
            ],
            [
                2,
                "year"
            ],
            [
                2,
                "theme"
            ],
            [
                2,
                "host city id"
            ],
            [
                2,
                "hosts"
            ],
            [
                3,
                "competition id"
            ],
            [
                3,
                "farm id"
            ],
            [
                3,
                "rank"
            ]
        ],
        "column_names_original": [
            [
                -1,
                "*"
            ],
            [
                0,
                "City_ID"
            ],
            [
                0,
                "Official_Name"
            ],
            [
                0,
                "Status"
            ],
            [
                0,
                "Area_km_2"
            ],
            [
                0,
                "Population"
            ],
            [
                0,
                "Census_Ranking"
            ],
            [
                1,
                "Farm_ID"
            ],
            [
                1,
                "Year"
            ],
            [
                1,
                "Total_Horses"
            ],
            [
                1,
                "Working_Horses"
            ],
            [
                1,
                "Total_Cattle"
            ],
            [
                1,
                "Oxen"
            ],
            [
                1,
                "Bulls"
            ],
            [
                1,
                "Cows"
            ],
            [
                1,
                "Pigs"
            ],
            [
                1,
                "Sheep_and_Goats"
            ],
            [
                2,
                "Competition_ID"
            ],
            [
                2,
                "Year"
            ],
            [
                2,
                "Theme"
            ],
            [
                2,
                "Host_city_ID"
            ],
            [
                2,
                "Hosts"
            ],
            [
                3,
                "Competition_ID"
            ],
            [
                3,
                "Farm_ID"
            ],
            [
                3,
                "Rank"
            ]
        ],
        "column_types": [
            "text",
            "number",
            "text",
            "text",
            "number",
            "number",
            "text",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "number",
            "text",
            "number",
            "text",
            "number",
            "number",
            "number"
        ],
        "db_id": "farm",
        "foreign_keys": [
            [
                20,
                1
            ],
            [
                23,
                7
            ],
            [
                22,
                17
            ]
        ],
        "primary_keys": [
            1,
            7,
            17,
            22
        ],
        "table_names": [
            "city",
            "farm",
            "farm competition",
            "competition record"
        ],
        "table_names_original": [
            "city",
            "farm",
            "farm_competition",
            "competition_record"
        ]
    }"""
input = 'Show the official names of the cities that have hosted more than one competition.'
main_task = f'Write a SQL include SELECT, FROM, WHERE to answer the specific QUESTION knowing Table = {table}\nQUESTION = {input}'
prompter = LanguageModelPrompter(lm, f"Main Task: {main_task}", json_format=False)
print(f"{main_task} {input}")




Write a SQL include SELECT, FROM, WHERE to answer the specific QUESTION knowing Table =  {
        "column_names": [
            [
                -1,
                "*"
            ],
            [
                0,
                "city id"
            ],
            [
                0,
                "official name"
            ],
            [
                0,
                "status"
            ],
            [
                0,
                "area km 2"
            ],
            [
                0,
                "population"
            ],
            [
                0,
                "census ranking"
            ],
            [
                1,
                "farm id"
            ],
            [
                1,
                "year"
            ],
            [
                1,
                "total horses"
            ],
            [
                1,
                "working horses"
            ],
            [
                1,
               

In [2]:
split_prompt = prompter.split_prompt({"state": f"Start\nInput", "num_split": 3, "current": input})
print(split_prompt)

<Instruction>STEP: Divide the current task into three subtasks by focusing on the following aspects:
1. Subtask 1: Identify the cities that have hosted competitions.
2. Subtask 2: Count the number of competitions hosted by each city.
3. Subtask 3: Select the cities that have hosted more than one competition. 
Answer each subtask seperately, each answer in |answer| and seperated by \t without any addtional text or thoughts.</Instruction>
Input: Show the official names of the cities that have hosted more than one competition.
Answer:


In [3]:
split_result = lm.get_response_texts(lm.query(split_prompt, 1, main_task))[0]
print(split_result)


|Subtask 1|SELECT "Official_Name", "Host_city_ID" FROM "farm competition"|
|Subtask 2|SELECT "Host_city_ID", COUNT("Competition_ID") AS "Num_Competitions" FROM "farm competition" GROUP BY "Host_city_ID"|
|Subtask 3|SELECT "Official_Name" FROM "city" WHERE "City_ID" IN (SELECT "Host_city_ID" FROM "farm competition" GROUP BY "Host_city_ID" HAVING COUNT("Competition_ID") > 1)|


In [4]:
aggregate_prompt = prompter.aggregate_prompt({"state": """STEP: Divide the current task into three subtasks by focusing on the following aspects:
1. Subtask 1: Identify the cities that have hosted competitions.
2. Subtask 2: Count the number of competitions hosted by each city.
3. Subtask 3: Select the cities that have hosted more than one competition. """, "current":split_result})


In [5]:
print(aggregate_prompt)

<Instruction>STEP: Combine the results of Subtask 1 and Subtask 3 by using a JOIN operation to retrieve the official names of the cities that have hosted more than one competition. Use the city table to match the city IDs obtained from Subtask 1 with the official names of the cities. This will provide the final result showing the official names of the cities that have hosted more than one competition. 
Only answer the result, each answer in |answer| and seperated by \t without any addtional text or thoughts.</Instruction>
Input: |Subtask 1|SELECT "Official_Name", "Host_city_ID" FROM "farm competition"|
|Subtask 2|SELECT "Host_city_ID", COUNT("Competition_ID") AS "Num_Competitions" FROM "farm competition" GROUP BY "Host_city_ID"|
|Subtask 3|SELECT "Official_Name" FROM "city" WHERE "City_ID" IN (SELECT "Host_city_ID" FROM "farm competition" GROUP BY "Host_city_ID" HAVING COUNT("Competition_ID") > 1)|
Answer:


In [6]:
lm.get_response_texts(lm.query(aggregate_prompt, 1, main_task))[0]

'|official name|\n|New York City|'